In [1]:
from sentence_transformers import SentenceTransformer, util
from PIL import Image
import requests

# Load a pre-trained CLIP model
model = SentenceTransformer('clip-ViT-B-32')

d:\HCMUS_ComputerScience\ComputationalThinking\TeamProject\reactProject\CLIP_Model\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


In [2]:
## declare constant and necessary variables
DB_path = "../DataCollector/result/places.db"
DB_name = "places"

In [3]:
import json
import sqlite3
from typing import List
import torch

def parseList(image_list):
    print(type(image_list))
    return json.loads(image_list)

def getImages(id):
    "Return a list of PIL images"
    conn = sqlite3.connect(DB_path)
    cursor = conn.cursor()
    cursor.execute(f"SELECT images FROM {DB_name} WHERE rowid = ?", (id,))
    image_url = cursor.fetchone()[0]
    image_url = parseList(image_url)
    pil_image = []
    for url in image_url:
        print(url)
        pil_image.append(Image.open(requests.get(url, stream=True).raw))

    return pil_image

def embedImageList(images):
    if len(images) == 0:
        return torch.zeros(1, 512)
    print("len(images) = ",len(images))
    encoding = [model.encode(image) for image in images]
    print(type(encoding[0]))
    # list of numpy.ndarray
    return sum(encoding) / len(encoding)
    

def embedText(text):
    return model.encode(text)

def getSimilarity(imageList, text):
    image_emb = embedImageList(imageList)
    text_emb = embedText(text)
    return util.cos_sim(image_emb, text_emb)

In [6]:
sample_text = "Đồ ăn ngon"
sample_images = [getImages(i) for i in range(2, 3)]

print(sample_images)


# my_embeddings = model.encode(sample_text)
print([getSimilarity(image, sample_text) for image in sample_images])

<class 'str'>
https://lh3.googleusercontent.com/p/AF1QipOSisTfpcYNbHV-0X3CB0E7pedEkh10avYId7eI=s0
https://lh3.googleusercontent.com/geougc-cs/AMBA38t2Kmr5_XJSv2PmDO10nabcToVMxqXZwW_M3PM9su_8PO1ED7uHBVqxnIGzYii6adp4DKOc07hYToYMGDCTK0A3sAnVZ0OcbUO7mh2w_5YBiC8MdqHRYBeFKFKBpXXethGqvS89=s0
https://lh3.googleusercontent.com/geougc-cs/AMBA38v8qd5Fo9KkbRzRKrYOOmP0JZxnFWPrjFUB2mbsDD04wT1pUBB01TM531RbG3_ZpfpUGoJgt2bvWrR0k5fFq7jkd3sp2m60IGig0JZJXf_wFPc2ANdfgFi1GH8-L_Wlg3dm-r9TpA=s0
[[<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=1080x1440 at 0x288D4B52910>, <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=3024x4032 at 0x288D4B3CB50>, <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=4032x3024 at 0x288D4A0E710>]]
len(images) =  3
<class 'numpy.ndarray'>
[tensor([[0.2461]])]


In [5]:
import faiss

# Define the path to your .bin file
index_file_path = "test_images.bin"

# Load the index from the file
try:
    index = faiss.read_index(index_file_path)
    print(f"Index loaded successfully from {index_file_path}")
    print(f"Index type: {type(index)}")
    print(f"Total number of vectors in the index: {index.ntotal}")
    for vector_id in range(10):
        # Returns a numpy array with shape (dimension,)
        vec = index.reconstruct(vector_id) 

        print(f"Vector ID {vector_id} first value: {vec[:10]}")
        print(f"Shape of the vector: {vec.shape}")

except RuntimeError as e:
    print(f"Error loading index: {e}")
    print("Ensure the file path is correct and the file is a valid FAISS index file.")


Index loaded successfully from test_images.bin
Index type: <class 'faiss.swigfaiss_avx2.IndexFlatL2'>
Total number of vectors in the index: 10
Vector ID 0 first value: [-0.06810567  0.01319423  0.34572196 -0.15017727  0.56712866 -0.2609901
 -0.11243334  0.19935876  0.6692451   0.04950246]
Shape of the vector: (512,)
Vector ID 1 first value: [-0.05430037  0.08203101  0.32165593 -0.00292639  0.6371502  -0.40463907
  0.13588804  0.19987547  0.43858612  0.33127224]
Shape of the vector: (512,)
Vector ID 2 first value: [-0.7186376  -0.06840184  0.16661792  0.4748946   0.07506506 -0.295602
  0.5527995   0.5271174  -0.2700316   0.43091565]
Shape of the vector: (512,)
Vector ID 3 first value: [-0.4066123  -0.24284181  0.3125778   0.3667134   0.17926046 -0.46689886
  0.17147157 -0.09994288 -0.648865    0.29288262]
Shape of the vector: (512,)
Vector ID 4 first value: [-0.599694   -0.00558543  0.3761969   0.46527785  0.27332696 -0.48122376
  0.2110529  -0.38230103 -0.47607696  0.15118171]
Shape of